In [2]:
import pandas as pd
from thunderpack import ThunderReader
from openpyxl import load_workbook

In [3]:
#get list of patients with missing discharge date from excel file
#include patient ID and encounter ID in the list
df = pd.read_excel('/home/cdac-c-15/Desktop/cardiac_filtered_orginal.xlsx')

missingDataPIDs = df[df['HospitalDischargeDTS'].isna()]['BDSPPatientID'].tolist()
missingDataEIDs = df[df['HospitalDischargeDTS'].isna()]['BDSPEncounterID'].tolist()
dfNoDischargeDate = pd.DataFrame({
    'PatientID': missingDataPIDs,
    'EncounterID': missingDataEIDs
})
print(len(dfNoDischargeDate))
dfNoDischargeDate.head()

2523


,PatientID,EncounterID
0,111249291,1565487169
1,111249291,1565487169
2,111249291,1565487169
3,111249291,1565487169
4,111249291,1565487169


In [ ]:
contains_pair = ((dfNoDischargeDate['PatientID'] == 111227152) & (dfNoDischargeDate['EncounterID'] == 533711015)).any()
print(f"{contains_pair}")

#checking that the list contains the correct pairs. true for 2 different IDS
#bad ids check? passed for both lists
#good ids check? passed for both lists


In [4]:
# create list of patients that has no duplicates 
noDischPID = dfNoDischargeDate['EncounterID']
noDischPID = dfNoDischargeDate.drop_duplicates()
print(len(noDischPID))
noDischPID.head()

491


,PatientID,EncounterID
0,111249291,1565487169
5,111317668,1602997059
14,111317668,1602996931
20,111504344,1625646484
29,111701935,1626494441


In [5]:
reader = ThunderReader('/media/cdac-c-15/Thunderpack/Dropbox/zz_EHR_Thunderpacks/MGB/thunderpack_adt_1m_MGB/')
print(len(list(reader.keys())))

47


In [6]:
# could use adt (transfer notes)
# 
# FIXME:this does not work, adt partition is wrong

dfs = []
for i in range(1, (len(list(reader.keys())) + 1)):
    df = reader[f'adt_main_partition_{i}']
    dfc = df[df['BDSPEncounterID'].isin(noDischPID['EncounterID'])]
    dfs.append(df)

In [7]:
filtered_icd_df = pd.concat(dfs, axis=0, ignore_index=True)     #puts data into rows and columns
print(len(df))      #print the number of notes found with those ICD codes
filtered_icd_df.head()      #print the first few lines of the filtered list of notes

295931


,BDSPPatientID,BDSPEncounterID,InEventDSC,OutEventDSC,SpecialtyNM,ReportGrouper10DSC,PatientClassDSC,TransferInDTS,TransferOutDTS,BDSPLastModifiedDTS
0,119675700,13294293869,Hospital Outpatient,Discharge,Radiology,Hospital Outpatient,Outpatient,2016-11-28 07:10:00.0000000,2016-11-28 07:27:00.0000000,2022-07-28 22:19:00.3200000
1,117444816,13290108151,Hospital Outpatient,Discharge,Pathology,Virtual Department,Outpatient,2015-03-09 08:41:00.0000000,2015-03-09 23:59:00.0000000,2022-07-28 22:19:00.3200000
2,120024142,13253605616,Transfer In,Transfer Out,Obstetrics,Virtual Department,Inpatient,2015-03-23 10:04:00.0000000,2015-03-23 10:05:00.0000000,2022-07-28 22:19:00.3200000
3,120507159,13233501255,Hospital Outpatient,Discharge,Radiology,Hospital Outpatient,Outpatient,2016-08-25 11:00:00.0000000,2016-08-25 23:59:00.0000000,2022-07-28 22:19:00.3200000
4,112546413,13260004012,Hospital Outpatient,Discharge,Lab,Hospital Outpatient,Outpatient,2015-07-27 08:37:00.0000000,2015-07-27 23:59:00.0000000,2022-07-28 22:19:00.3200000


In [11]:
filtered_icd_df = filtered_icd_df[filtered_icd_df['OutEventDSC'] == 'Discharge']
filtered_icd_df['TransferOutDTS'] = pd.to_datetime(filtered_icd_df['TransferOutDTS'])
filtered_icd_df.head()

In [ ]:
#merge dfs to make dfs contain missing data
dfNoDischargeDate = pd.merge(filtered_icd_df, dfNoDischargeDate, left_on=['BDSPPatientID', 'BDSPEncounterID'], right_on=['PatientID', 'EncounterID'], how='inner')
dfNoDischargeDate.head()

NameError: name 'filtered_icd_df' is not defined

In [ ]:
excel_df = pd.read_excel('/home/cdac-c-15/Desktop/cardiac_filtered_orginal.xlsx')

In [ ]:
#merge excel sheet with existing data
# Assuming 'PatientID' and 'EncounterID' are columns in both DataFrames
# and we want to update the 'Data_Column' in the Excel DataFrame

for index, row in dfNoDischargeDate.iterrows():
    id_value = row['PatientID']
    encounter_id_value = row['EncounterID']
    
    # Find the index in the original Excel DataFrame
    match_index = excel_df[(excel_df['PatientID'] == id_value) & (excel_df['EncounterID'] == encounter_id_value)].index
    
    # Update the Excel DataFrame with the new data
    if not match_index.empty:
        excel_df.loc[match_index, 'DischargeDate'] = row['NewDischargeDate']  # Replace 'Data_Column' with the actual column name to update

print("Updated Excel DataFrame:")
print(excel_df)